# Deploy R Shiny Dashboard App

### Introduction

Now that we have built the machine learning pipeline, stored and deployed it using [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net) , we can use the pipeline to ingest new data, prep it and score it. 

In this notebook we will:

* Programmatically get the ID's for the deployment space that was created in the **1-model-training** notebook.
* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard

In [8]:
import os
import pandas as pd
import datetime
import random
import string
from ibm_watson_machine_learning import APIClient


token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

client = APIClient(wml_credentials)


### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in 1-model-training. 
We use the space name that was used when creating the deployments as specified below. 

In [9]:
space_name = 'Utilities Customer Attrition Space'

Get the space we are working in, which is found using the name that were hardcoded in **1-model-training**. 
If the user would like to use a different space manually set the **space_id**.

Set the space as the default space for working.

In [10]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

# Deploy Shiny App

In this section we will complete the steps to deploy a Shiny Dashboard in Cloud Pak for Data. The app can be deployed in a similar way to models and functions, using the `ibm-watson-machine-learning` package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

In [11]:
r_shiny_deployment_name='Utilities-Customer-Attrition-Shiny-App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [12]:
rshiny_spec_uid=client.software_specifications.get_id_by_name("rstudio_r4.2")
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Utilities_Customer_Attrition_Shiny_assets",
    client.shiny.ConfigurationMetaNames.SOFTWARE_SPEC_UID:rshiny_spec_uid,
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/utilities-customer-attrition-prediction-analytics-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are:
- Anyone with the link 
- Authenticated users 
- Collaborators in this deployment space

In [13]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Utilities Customer Attrition dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1},
    client.deployments.ConfigurationMetaNames.SERVING_NAME:r_shiny_deployment_name.lower().replace('-','')[:30]+''.join(random.choice(string.digits) for _ in range(3))
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '473b1667-d416-4b53-8c85-fffbd4ddda2d' started

#######################################################################################


initializing..........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b6cacb99-dc22-491c-bb7d-d1ec502c7767'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select **Deployments -> Spaces -> Utilities Customer Attrition Space -> Deployments -> Utilities-Customer-Attrition-Shiny-App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [14]:
print("{HOSTNAME}"+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')

{HOSTNAME}/ml/v4/deployments/b6cacb99-dc22-491c-bb7d-d1ec502c7767/r_shiny


**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2023. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.** <br>